## Tutorial for processing stellar luminosities from Galacticus HDF5 output.

In [1]:
import numpy as np

# Load GalacticusHDF5 class for reading/writing of Galacticus HDF5 files.
from galacticus.io import GalacticusHDF5
# Load StellarLuminosities class for computing/extracting stellar luminosity datasets from HDF5 file
from galacticus.StellarLuminosities import StellarLuminosities

In [2]:
# Load Galacticus HDF5 file
ifile = "../galacticus/data/exampleRuns/galacticus.snapshotExample.hdf5"
GH5 = GalacticusHDF5(ifile,'r')

In [3]:
# Print available redshifts
for i in range(len(GH5.outputs.z)):
    print(str(GH5.outputs.iout[i])+" "+GH5.outputs.name[i]+" z = "+str(GH5.outputs.z[i]))

1 Output1 z = 2.0
2 Output2 z = 1.0
3 Output3 z = 0.0


In [4]:
# Pass Galacticus HDF5 class to Stellar Luminosities class
STELLAR = StellarLuminosities(GH5)

In [5]:
# List all stellar luminosities available at z = 1.4 and print top 10.
z = 1.4
availableLuminosities = STELLAR.availableLuminosities(z)
print(availableLuminosities[:10])

['diskLuminositiesStellar:HeliumContinuum:observed:z1.0000', 'diskLuminositiesStellar:HeliumContinuum:observed:z1.0000:recent', 'diskLuminositiesStellar:HeliumContinuum:rest:z1.0000', 'diskLuminositiesStellar:HeliumContinuum:rest:z1.0000:recent', 'diskLuminositiesStellar:Lyc:observed:z1.0000', 'diskLuminositiesStellar:Lyc:observed:z1.0000:recent', 'diskLuminositiesStellar:Lyc:rest:z1.0000', 'diskLuminositiesStellar:Lyc:rest:z1.0000:recent', 'diskLuminositiesStellar:OxygenContinuum:observed:z1.0000', 'diskLuminositiesStellar:OxygenContinuum:observed:z1.0000:recent']


The stellar luminosities datasets have the following nomenclature:

    <component>LuminositiesStellar:<filterName>:<frame>:z<redshift>(:recent)(:dust<dustOptions>)
where:

    <component> = the component of the galaxy ('disk', 'spheroid' or 'total')
    <filterName> = the name of the filter used to compute this luminosity (e.g. SDSS r-band)
    <frame> = the frame of the luminosity ('rest' or 'observed')
    <redshift> = the redshift of the snapshot output in which this galaxy is found (e.g 1.4000). Note that for lightcones this may be different to the redshift of the galaxy in the lightcone.
    <dustOptions> = if the luminosity has attenuated by interstellar this will consist of a string of commands describing the dust method that has been applied.


If *:recent* is present, the luminosity will correspond to star formation that has occured in the last **X** Gyrs.

In [6]:
# We can extract existing luminosities using the readGalaxies function in the GalacticusHDF5 class. For example:
luminosityName = availableLuminosities[0]
print("Reading "+luminosityName+"...")
lum = GH5.readGalaxies(z,props=[luminosityName])
print(lum[luminosityName][:10]) # Printing top 10 only

Reading diskLuminositiesStellar:HeliumContinuum:observed:z1.0000...
[ 268.61919176    7.76074644   63.85530689   26.96135918    9.1361605
   95.438968     27.47942891   38.36634648    6.77485493   34.27606443]


In [7]:
# Or we can use the getLuminosity function in the StellarLuminosities class:
arr = STELLAR.getLuminosity(luminosityName)
print(arr[:10])

[ 268.61919176    7.76074644   63.85530689   26.96135918    9.1361605
   95.438968     27.47942891   38.36634648    6.77485493   34.27606443]


In [8]:
# In a raw Galacticus HDf5 file luminosities are stored seperately for the disk and bule of a galaxy. However, we
# we can use the getLuminosity function to compute the total luminosity for the entire galaxy. This function simply
# sums the luminosites for the two components. Here are what the names look like for the different components:
print("Disk luminosity name = "+luminosityName)
sphereName = luminosityName.replace("disk","spheroid")
print("Bulge luminosity name = "+sphereName)
totalName = luminosityName.replace("disk","total")
print("Total luminosity name = "+totalName)

Disk luminosity name = diskLuminositiesStellar:HeliumContinuum:observed:z1.0000
Bulge luminosity name = spheroidLuminositiesStellar:HeliumContinuum:observed:z1.0000
Total luminosity name = totalLuminositiesStellar:HeliumContinuum:observed:z1.0000


In [9]:
# WE can comute the total luminosity using the getLuminosity function:
totalLum = STELLAR.getLuminosity(totalName)
print totalLum[:10]

[ 268.61919176    7.90182634   63.85530689   26.96135918    9.1361605
  101.00718134   27.47942891   38.36634648    6.77485493   34.27606443]


In [10]:
# Which is the same as adding the disk and bulge components manually:
diskLum = STELLAR.getLuminosity(luminosityName)
sphereLum = STELLAR.getLuminosity(sphereName)
lum = diskLum + sphereLum
print(lum[:10])

[ 268.61919176    7.90182634   63.85530689   26.96135918    9.1361605
  101.00718134   27.47942891   38.36634648    6.77485493   34.27606443]


In [11]:
# The luminsoities are stored in units of AB luminosities, such that we can directly conver them to AB 
# (or Vega) magnitudes. We can convert the luminosities to solar luminosities:
from galacticus.constants import luminosityAB,luminositySolar
print(lum[:10]*luminositySolar/luminosityAB)

[  2.30910773e+15   6.79257807e+13   5.48913806e+14   2.31765581e+14
   7.85363798e+13   8.68279225e+14   2.36219018e+14   3.29805279e+14
   5.82380946e+13   2.94644344e+14]


In [12]:
GH5.close()